# Algorithm Used: KNN(k nearest neighbours) is a supervised learning method, where the value of K is the number of nearest neighbors. where generally the value of K is odd if the number of class is 2.
# If K > 1: Calculates the Distance, Finds the closest neighbors and votes for the labels. The class with the most votes is then assigned that label to the class with no labels

# We are using the MFCC(Mel Frequency Cepstral Coefficients) features from the Python_Speech_Features library, which are used in automatic audio and speech recognition studies. What MFCC does is extract the Low frequencies from the audio file and the time frequencies of the audio file. By using these frequencies the model classifies the file according to the output required. Here we use it to classify the genres.

In [1]:
from python_speech_features import mfcc #used to get the MFCC features.
import scipy.io.wavfile as wav #used to read the wav files.
import numpy as np

import os
import pickle
import random
import operator

# function to get the distance between feature vectors and find neighbours

In [2]:
def getNeighbors(trainingSet, instance, k): #here the top K neighbors will be used to identify the label or the class of the input file.
    distances = []
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

# identify the class of the instance

In [3]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)

    return sorter[0][0]

# function to evaluate the model

In [4]:
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (1.0 * correct) / len(testSet)

# Here we extract features from the dataset and dump these features into a binary .dat file “my.dat”.

# for each file the rate and the signature is extracted, then using the mfcc function the features are identified and based on these features the files are classified. These features are then dumped in the dat file.

In [5]:
directory = "E:/Computer/python/Music_Genre_Classification/Data/genres_original/"

# binary file where we will collect all the features extracted using mfcc (Mel Frequency Cepstral Coefficients) and will be stored in the dat file.
f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file) 
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            pass
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported.  in folder:  jazz  filename:  jazz.00054.wav


#### The above cell gives exception as the file: jazz.00054 is corrupted.

# Split the dataset into training and testing sets respectively in the ratio of 70:30, 70 for training and 30 for testing.

In [6]:

dataset = []

def loadDataset(filename, split, trSet, teSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('my.dat', 0.7, trainingSet, testSet)

# Function to calculate the distance between the features

In [7]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

# making predictions using KNN

In [8]:

leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.7087378640776699


# testing the code with external sample files from the test folder, a single file is selected and then the Genre is predicted of that file.

In [29]:
# URL used for the test files: https://uweb.engr.arizona.edu/~429rns/audiofiles/audiofiles.html.
test_dir = "E:/Computer/python/Music_Genre_Classification/Test/"
#test_file = test_dir + "test.wav" #classical
#test_file = test_dir + "test2.wav" #pop
#test_file = test_dir + "test4.wav" #country

#Extra Songs used.
test_file = test_dir + "test5.wav" #Eminem Song: Love the way you lie. Genre: Rap and Pop, but rap is not one of the classified Genre, so Pop is Predicted.
#test_file = test_dir + "test6.wav" #Dua Lipa Song: Be the One. Genre:Pop.


In [30]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

# File Test6 is different from the Hz range of the files we used earlier and hence we get the warning.

# After the file is loaded and tested, while getting the prediction to get the genre name, we use the dictionary below where the names of the genres are stored.

In [31]:
from collections import defaultdict
results = defaultdict(int)

directory = "E:/Computer/python/Music_Genre_Classification/Data/genres_original/"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

## Here we predict the test the model using one the 6 test files.

In [32]:
pred = nearestClass(getNeighbors(dataset, feature, 5))
print(results[pred])

pop
